## !doc + CONTEXT (2030)

> https://www.gov.uk/guidance/find-and-use-data-on-public-electric-vehicle-chargepoints

>  API doc: https://chargepoints.dft.gov.uk/api/help

> NOTE - direct csv, json and xml downloads possible at: 
https://chargepoints.dft.gov.uk/api/retrieve/registry/
https://chargepoints.dft.gov.uk/api/retrieve/registry/format/json
https://www.gov.uk/guidance/find-and-use-data-on-public-electric-vehicle-chargepoints (download CSV from this page)
    

In [ ]:
# imports

import requests
import pandas as pd
# pd.options.display.max_columns = 200


## Query the API and filter the result

In [ ]:
url = 'https://chargepoints.dft.gov.uk/api/retrieve/registry/format/json'

req = requests.get(url)
response = req.json()

In [ ]:
response.keys()

dict_keys(['Scheme', 'ChargeDevice'])

In [ ]:
# how many items inside 'ChargeDevice

len(response['ChargeDevice'])

12330

In [ ]:
# assign this to a variable
data = response['ChargeDevice']

# look at one instance of the data
data[9]

In [ ]:
rows = []

target = 'Cardiff'

for x in data:
  
    if x['ChargeDeviceLocation']['Address']['PostTown'] == target:
        title = x['ChargeDeviceName']
        lat = x['ChargeDeviceLocation']['Latitude']
        long = x['ChargeDeviceLocation']['Longitude']
        place = x['ChargeDeviceLocation']['Address']['PostTown']
        code = x['ChargeDeviceLocation']['Address']['PostCode']
        owned_by = x['DeviceOwner']['OrganisationName']
        telephone = x['DeviceOwner']['TelephoneNo']
        loc = x['LocationType']
    
        rows.append({"name": title, "latitude": lat, "longitude": long, "town": place, "postcode": code,"owner": owned_by, "phone": telephone, "location_type": loc})


In [ ]:
## assemble the df

df = pd.DataFrame(rows)
df.shape

(30, 8)

## select for all of Wales

In [ ]:
rows = []
wpa = ['CF', 'NP', 'SA', 'LD', 'SY', 'LL']

for x in data:
    
    response_code = x['ChargeDeviceLocation']['Address']['PostCode']   
    if response_code != None and response_code[:2] in wpa:
        
        title = x['ChargeDeviceName']
        lat = x['ChargeDeviceLocation']['Latitude']
        long = x['ChargeDeviceLocation']['Longitude']
        place = x['ChargeDeviceLocation']['Address']['PostTown']
        county = x['ChargeDeviceLocation']['Address']['County']
        code = x['ChargeDeviceLocation']['Address']['PostCode']
        connec = len(x['Connector'])
        long_loc = x['ChargeDeviceLocation']['LocationLongDescription']
        owned_by = x['DeviceOwner']['OrganisationName']
        website = x['DeviceOwner']['Website']
        telephone = x['DeviceOwner']['TelephoneNo']
        working = x['ChargeDeviceStatus']
        date_c = x['DateCreated']
        date_u = x['DateUpdated']
        subs = x['SubscriptionRequiredFlag']
        subs_details = x['SubscriptionDetails']
        loc = x['LocationType']
        twentyfourhrs = x['Accessible24Hours']


        rows.append({"name": title, "latitude": lat, "longitude": long, "town": place, "county": county, 
                     "postcode": code, "nb_connectors": connec, "address": long_loc, "owner": owned_by, "website": website, 
                     "phone": telephone, "date_created":date_c, "date_updated": date_u, 
                     "subscription": subs, "subscription_details": subs_details, "location_type": loc, 
                    "24hours": twentyfourhrs, "working_order":working})

In [ ]:
wdf = pd.DataFrame(rows)
wdf[:3]

,name,latitude,longitude,town,county,postcode,nb_connectors,address,owner,website,phone,date_created,date_updated,subscription,subscription_details,location_type,24hours,working_order
0,Easthope Car Park Church Stretton,52.537630,-2.807690,Church Stretton,Shropshire,SY6 6BX,2,On RHS of car park before the Coop. Parking ch...,Church Stretton District Council,http://www.churchstretton.co.uk,01694 722113,2014-09-24 14:41:27,2016-03-15 11:09:18,False,None,Other,True,In service
1,Cletwr car park,52.512613,-3.975689,Machynlleth,Powys,SY20 8PN,3,None,Chargemaster (POLAR),https://www.chargemasterplc.com,01582 400331,2019-11-08 11:34:07,2020-04-26 14:39:35,True,None,Retail car park,False,In service
2,NCP Greyfriars,51.483101,-3.175770,Cardiff,NA,CF10 3AD,2,None,Chargemaster (POLAR),https://www.chargemasterplc.com,01582 400331,2020-05-04 10:33:48,2020-05-04 10:33:48,True,None,Service station,False,In service


## parameters for lat, long + radius

In [ ]:
# see documentation

format_type = 'json'
latitude = '51.545581'
longitude = '-0.077301'
radius = '10'

query = f'http://chargepoints.dft.gov.uk/api/retrieve/registry/format/{format_type}/lat/{latitude}/long/{longitude}/dist/{radius}'

req = requests.get(query)
req

<Response [200]>

In [ ]:
response = req.json()
response

In [ ]:
len (response)

2

In [ ]:
response.keys()

dict_keys(['Scheme', 'ChargeDevice'])

In [ ]:
data = response['ChargeDevice']

In [ ]:
len(data)

3458

In [ ]:
data[4]

{'AccessRestrictionDetails': '',
 'AccessRestrictionFlag': False,
 'Accessible24Hours': False,
 'Attribution': 'Source London',
 'Bearing': None,
 'ChargeDeviceId': 'f8411c202a5cf1e26a0eb50979857d44',
 'ChargeDeviceLocation': {'Address': {'BuildingName': '',
   'BuildingNumber': '321A',
   'Country': 'gb',
   'County': 'Greater London',
   'DependantLocality': None,
   'DoubleDependantLocality': None,
   'PostCode': 'N11 1BA',
   'PostTown': 'New Southgate',
   'Street': '',
   'SubBuildingName': None,
   'Thoroughfare': 'Bowes Road',
   'UPRN': None},
  'Latitude': '51.616451',
  'LocationLongDescription': '',
  'LocationShortDescription': None,
  'Longitude': '-0.132860'},
 'ChargeDeviceManufacturer': None,
 'ChargeDeviceModel': '',
 'ChargeDeviceName': 'Arnos Grove Station Car Park 1',
 'ChargeDeviceRef': 'SRC_LDN60196',
 'ChargeDeviceStatus': 'In service',
 'ChargeDeviceText': None,
 'Connector': [{'ChargeMethod': 'Single Phase AC',
   'ChargeMode': '3',
   'ChargePointStatus': 'In